# Configuration

In [1]:
# General configuration
import os

# data_directory: str
#     Path to a directory to store data.
data_directory = '.'

# install_missing_packages: bool
#     A flag indicating if missing packages should be automatically installed
install_missing_packages = True

# use_conda: bool
#     A flag indicating if conda should be used for software installation.
#     If False, pip will be used. The default is to use conda if jupyter
#     is run in a conda environment.
use_conda = 'CONDA_EXE' in os.environ

## Checking for missing packages

In [2]:
import importlib

def check_package(package, pip_pkg: str = None, conda_pkg: str = None):
    """Check if a given package is installed. If missing install
    it (if global flag `install_missing_packages` is True) either with
    pip or with conda (depending on `use_conda`).
    """
    if importlib.util.find_spec(package) is not None:
        return  # ok, package is already installed

    if not install_missing_packages:
        raise RuntimeError(f"{package} is not installed!")

    if use_conda:
        import conda.cli
        conda.cli.main('conda', 'install',  '-y', conda_pkg or package)
    else:
        import subprocess
        import sys            
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_pkg or package])
        
# This is to exit cells without error tracebacks (cosmetic purpose)
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Creating the required environment (skip if already done)

Running the following cell will create a file graphs.yml that can be used to setup a conda environment containing the required packages. If you already downloaded the file from my GitHub, skip the next cell and create the env directly from it.

In [3]:
%%writefile graphs.yml
name: graphs
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.6
  - jupyter
  - imageio
  - imageio-ffmpeg
  - matplotlib
  - scikit-image
  - opencv
  - networkx
  - pandas
  - statsmodels
  - scikit-learn

Writing graphs.yml


### Environment Creation
To create the environment, open the terminal, go to the directory where you stored the graphs.yml file (the directory of the notebook) and type
`conda env create -f graphs.yml`
After running this command you have to activate the environment (Linux/MacOS: `conda activate graphs`, Windows: activate graphs) and then reopen the notebook in that environment.
### JypiterNotebook integration
Now comes the step to set this conda environment, to have the enviroment as a kernel in your jypiterNotebook. On the same terminal into which yoou have activated your enviroment type:
- `conda install -c anaconda ipykernel`
- `python -m ipykernel install --user --name=graphs`
- Reload your notebook 
- Select graphs as your kernel 


### Imports and directory information

In [5]:
import os
import cv2
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import glob
import scipy.cluster.vq as clusters
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import normalize
from pandas.plotting import autocorrelation_plot as AC_plot 
from statsmodels.graphics import tsaplots
from statsmodels.tsa.stattools import acf
from skimage.filters import gaussian
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib.colors import LinearSegmentedColormap

In [6]:
OG_DATA_PATH = '/Users/tracysanchezpacheco/Documents/HumanAnalysis/HumanAnalysis'
GIT_DATA_PATH = '/Users/tracysanchezpacheco/Documents/HumanAnalysis/HumanAnalysis/Data Exploration'
GIT_PROCESSED_DATA_PATH = '/Users/tracysanchezpacheco/Documents/HumanAnalysis/HumanAnalysis/Results'
GIT_GRAPH_PATH = '/Users/tracysanchezpacheco/Documents/HumanAnalysis/HumanAnalysis/Results/Graphs/'
RESSOURCES_PATH = '/Users/tracysanchezpacheco/Documents/HumanAnalysis/HumanAnalysis/Resources/'

# Reset the Datapath since the data is not yet on Git, comment out if data is on Git 
DATA_PATH = GIT_DATA_PATH
PROCESSED_DATA_PATH = GIT_PROCESSED_DATA_PATH
GRAPH_DATA_PATH = GIT_GRAPH_PATH

# Getting the Folder without hidden files in ascending order 
DATA_FOLDER = sorted([f for f in os.listdir(DATA_PATH) if not f.startswith('.')], key=str.lower)
PROCESSED_DATA_FOLDER = sorted([f for f in os.listdir(PROCESSED_DATA_PATH) if not f.startswith('.')], key=str.lower)
GIT_PROCESSED_DATA_FOLDER = sorted([f for f in os.listdir(GIT_PROCESSED_DATA_PATH) if not f.startswith('.')], key=str.lower)


#houselist 
house_file = RESSOURCES_PATH + 'building_collider_list.csv'
try:
    houselist = pd.read_csv(house_file)
except:
    print('HouseList could not be loaded!')

In [6]:
DATA_FOLDER

['000_Expl_S_000_ET_1_1632396670,86291.json',
 '000_TaskDataFrame__EyeTracking1_1632662251,68487.json',
 '999_EyeValidation_1632408958.04931_TA.json',
 '999_PointingTask_Randomization_1632408793.09208.json',
 '999_TaskDataFrame_1632410484.07538.json',
 '999_TaskDataFrame_1632410493.17054_OnQuit.json',
 '999_TaskDataFrame__EyeTracking1_1632410484.06745.json',
 '999_TaskDataFrame__EyeTracking2_1632410493.17004_OnQuit.json']

### Extracting all subject IDs from the data folder

In [7]:
subIDs = []
for sub in DATA_FOLDER:
    if sub[0].isdigit():
        subIDs.append(int(sub[0:3])) #First four characters are numbers 
    else:
        pass
subIDs = np.unique(subIDs)
print(subIDs)

[  0 999]


In [8]:

landmarks = ['Castle-TaskBuilding_56',
             'Church-TaskBuilding_16',
             'HighSilo-TaskBuilding_49',
             'Windmill-TaskBuilding_10_1',
             'crane_1',
             'crane_2']

garages_to_stay = ['Garage_185', 
                   'Garage_224', 
                   'Garage_235',  
                   'Garage_86', 
                   'Garage_98']


# column name list for dataframe
col_names =  ['Session',
              'timeStampDataPointStart',
              'timeStampDataPointEnd',
              'hitObjectColliderName', 
              'ordinalOfHit',
              'BitMask',
              'hitPointOnObject.x',
              'hitPointOnObject.y',
              'hitPointOnObject.z',
              'hitObjectColliderBoundsCenter.x',
              'hitObjectColliderBoundsCenter.y',
              'hitObjectColliderBoundsCenter.z',
              'transformed_collidercenter_x',
              'transformed_collidercenter_y'
              'hmdPosition.x',
              'hmdPosition.y',
              'hmdPosition.z']


NoHit_dict = {'hitObjectColliderName': 'NoHit',
              'ordinalOfHit': '1'}


subject_cols = ['SubjectID',
                'Sessions',
                'ET_Sessions',
                'Total Rows Combined',
                'Removed Body Hits', 
                'Removed Graffiti Hits', 
                'Replaced NoHouse Hits',
                'DataLoss BitMask', 
                'DataLoss NoHits',
                'DataLoss Combined']

subject_data_df = pd.DataFrame(columns=subject_cols)
subject_data_df

,SubjectID,Sessions,ET_Sessions,Total Rows Combined,Removed Body Hits,Removed Graffiti Hits,Replaced NoHouse Hits,DataLoss BitMask,DataLoss NoHits,DataLoss Combined


In [9]:
  # Create empty dataframe for later concatenation
    complete_exploration_df = pd.DataFrame(columns = col_names)
    complete_exploration_df

,Session,timeStampDataPointStart,timeStampDataPointEnd,hitObjectColliderName,ordinalOfHit,BitMask,hitPointOnObject.x,hitPointOnObject.y,hitPointOnObject.z,hitObjectColliderBoundsCenter.x,hitObjectColliderBoundsCenter.y,hitObjectColliderBoundsCenter.z,transformed_collidercenter_x,transformed_collidercenter_yhmdPosition.x,hmdPosition.y,hmdPosition.z


In [ ]:
or ET_session in subject_data:
                s+=1

                # open the JSON file as dictionary
                with open(DATA_PATH + ET_session) as f:
                    try:
                        subject_session = json.loads(f.read())
                    except:
                        print("\tJSON file " + ET_session + " is not valid!")

,Session,timeStampDataPointStart,timeStampDataPointEnd,hitObjectColliderName,ordinalOfHit,BitMask,hitPointOnObject.x,hitPointOnObject.y,hitPointOnObject.z,hitObjectColliderBoundsCenter.x,hitObjectColliderBoundsCenter.y,hitObjectColliderBoundsCenter.z,transformed_collidercenter_x,transformed_collidercenter_yhmdPosition.x,hmdPosition.y,hmdPosition.z


In [15]:
folder_files

[]

In [ ]:
results = pd.read_json('Test.json')


# In[3]:


df = pd.DataFrame.from_records(results["trials"], columns = ['trialId', 'someRandomInformation',  'timeTrialMeasurementStarted', 'timeTrialMeasurementStopped', 'dataPoints'])
df


# In[4]:


df['dataPoints']


# In[52]:


datapoints = ['timeStampDataPointStart']


# In[8]:


new_df = pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in df['dataPoints']],ignore_index=True)


# In[9]:


new_df.head()


# In[5]:


#df2 = pd.json_normalize(results['trials'])
pd.json_normalize(df['dataPoints'])


# In[66]:


df2


# In[32]:


df2 = pd.DataFrame.from_records(df["dataPoints"], columns = ['eyePositionCombinedWorld'])


# In[40]:


df2 = df.dataPoints


# In[43]:


df2n = pd.json_normalize(df['dataPoints'])


# In[61]:


results.trials['dataPoints']